In [34]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [35]:
sns.set(color_codes=True)

In [36]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [40]:
other_df = pd.get_dummies(train['DetectedCamera']) 
other_df_test = pd.get_dummies(test['DetectedCamera'])

In [41]:
merged_df = pd.concat([other_df,train],1) 
merged_df_test = pd.concat([other_df_test,test],1)

In [43]:
le = LabelEncoder()
merged_df['SignFacing (Target)']=le.fit_transform(merged_df['SignFacing (Target)'])


In [44]:
train = merged_df 
test = merged_df_test

In [47]:
train_label = train['SignFacing (Target)']

training = train.drop(labels=['Id', 'DetectedCamera', 'SignFacing (Target)'], axis=1)
testing = test.drop(labels=['Id', 'DetectedCamera'], axis=1)

In [99]:
#It wont work. Dont know why
training['sin'] = np.sin(training['AngleOfSign'])
training['cos'] = np.cos(training['AngleOfSign'])

testing['sin'] = np.sin(testing['AngleOfSign'])
testing['cos'] = np.cos(testing['AngleOfSign'])

In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import xgboost as xgb


## Grid Search

In [209]:
# kfold = KFold(n_splits=3, random_state=7)
# model = xgb.XGBClassifier(objective='multi:softprob', nthread=-1, reg_lambda=1, seed=2017, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=4, colsample_bylevel=0.6, colsample_bytree=1)
# params_grid = { 'n_estimators':list([60,70,75,80,85]), 'reg_lambda':[5,6,7,8,9,10], 'subsample':[0.6,0.7,0.8,0.9,0.99]  }
# grid1 = GridSearchCV(estimator=model, param_grid=params_grid, n_jobs=-1, scoring='log_loss', cv=kfold)
# grid1.fit(training, train_label)

/home/clabuser/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/clabuser/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/clabuser/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/clabuser/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/clabuser/anaconda2/lib/python2.7/site-packages/sklearn/metrics/sco

GridSearchCV(cv=KFold(n_splits=3, random_state=7, shuffle=False),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=0.6, colsample_bytree=1,
       gamma=0.2, learning_rate=0.15, max_delta_step=0, max_depth=4,
       min_child_weight=4, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=2017, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [60, 70, 75, 80, 85], 'subsample': [0.6, 0.7, 0.8, 0.9, 0.99], 'reg_lambda': [5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='log_loss', verbose=0)

In [210]:
# print grid1.best_estimator_
# print grid1.best_params_
# print grid1.best_score_
# print grid1.best_index_

XGBClassifier(base_score=0.5, colsample_bylevel=0.6, colsample_bytree=1,
       gamma=0.2, learning_rate=0.15, max_delta_step=0, max_depth=4,
       min_child_weight=4, missing=None, n_estimators=75, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=5,
       scale_pos_weight=1, seed=2017, silent=True, subsample=0.9)
{'n_estimators': 75, 'subsample': 0.9, 'reg_lambda': 5}
-0.117412067479
63


In [153]:
# new_model = grid1.best_estimator_
# new_model.fit(training, train_label)
# prediction = new_model.predict_proba(testing)

## Prep For XGB

In [105]:
xtrain, xval, ytrain, yval = train_test_split(training,train_label,test_size=0.33,random_state=7)

In [106]:
preds = testing.columns
DMtraining = xgb.DMatrix(training, train_label, feature_names=preds)
DMtrain = xgb.DMatrix(xtrain,ytrain,feature_names=preds)
DMval = xgb.DMatrix(xval,yval,feature_names=preds)
DMtest = xgb.DMatrix(testing, feature_names=preds)

In [213]:
xgb_params={
    'eta':0.15,
    'max_depth':4,
    'colsample_bytree':1.0,
    'colsample_bylevel':0.6,
    'subsample' : 0.9,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'min_child_weight':1,
    'silent':1,
    'seed':2017,
    'nthread':4,
    'gamma' : 0.2,
    'num_class':4,
    'lambda':7,
    'alpha':0
}

In [214]:
watchlist = [(DMtrain, 'Train'), (DMval,'Validation')]
num_rounds= 2500
model = xgb.train(xgb_params, DMtrain, num_rounds, watchlist, early_stopping_rounds=80, verbose_eval=5)

[0]	Train-mlogloss:1.12573	Validation-mlogloss:1.12645
Multiple eval metrics have been passed: 'Validation-mlogloss' will be used for early stopping.

Will train until Validation-mlogloss hasn't improved in 80 rounds.
[5]	Train-mlogloss:0.519184	Validation-mlogloss:0.522277
[10]	Train-mlogloss:0.287542	Validation-mlogloss:0.293147
[15]	Train-mlogloss:0.187387	Validation-mlogloss:0.195171
[20]	Train-mlogloss:0.140496	Validation-mlogloss:0.150076
[25]	Train-mlogloss:0.118462	Validation-mlogloss:0.129389
[30]	Train-mlogloss:0.107003	Validation-mlogloss:0.119262
[35]	Train-mlogloss:0.100625	Validation-mlogloss:0.113975
[40]	Train-mlogloss:0.097209	Validation-mlogloss:0.111746
[45]	Train-mlogloss:0.094894	Validation-mlogloss:0.110287
[50]	Train-mlogloss:0.09314	Validation-mlogloss:0.109554
[55]	Train-mlogloss:0.091819	Validation-mlogloss:0.108996
[60]	Train-mlogloss:0.090355	Validation-mlogloss:0.108445
[65]	Train-mlogloss:0.08937	Validation-mlogloss:0.108158
[70]	Train-mlogloss:0.088359	Va

In [215]:
model = xgb.train(xgb_params, DMtraining,num_boost_round=int(82/95))
prediction = model.predict(DMtest)

In [216]:
subm_file = pd.read_csv('sample_submission.csv')

In [217]:
subm_file['Id'] = test['Id']
subm_file['Front'] = prediction[:,0]
subm_file['Left'] = prediction[:,1]
subm_file['Rear'] = prediction[:,2]
subm_file['Right'] = prediction[:,3]

In [156]:
subm_file.to_csv('subm_new3.csv', index=None)